In [130]:
from gurobipy import Model, quicksum, GRB

In [131]:
Plnt = {'plnt'}
DC = set(range(1,4))
Cust = DC.copy()
TRoute = {(i,j) for i in Plnt for j in DC}
DRoute = {(j,k) for j in DC for k in Cust}
Route = T_Route | D_Route

T = 2
Period = list(range(1, T+1))

In [134]:
Route

{('plnt', 1),
 ('plnt', 2),
 ('plnt', 3),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 1),
 (3, 2),
 (3, 3)}

In [135]:
del_cost = {(j, k):10 for j,k in DRoute}
demand = {(k,t):10 for k in Cust for t in Period}
run_dc_cost = {j:1000 for j in DC}

In [136]:
model = Model()

In [137]:
x,y = {}, {}

for j in DC:
    for k in Cust:
        for t in Period:
            x[j,k,t] = model.addVar(vtype="C", name=f'x[{j},{k},{t}]')

for j in DC:
    for t in Period:
        y[j,t] = model.addVar(vtype="B", name=f'y[{j},{t}]')

model.update()

In [138]:
Cust_Demand_Cons, DC_Running_Cons = {}, {}

for k in Cust:
    for t in Period:
        Cust_Demand_Cons[k,t] = model.addConstr(
            quicksum(x[j,k,t] for j in DC)
            ==
            demand[k,t]
        )

for j in DC:
    for k in Cust:
        for t in Period:
            DC_Running_Cons[j,k,t] = model.addConstr(
                x[j,k,t]
                <=
                demand[k,t] * y[j,t]
            )
    
model.update()

In [139]:
model.setObjective(
    quicksum(del_cost[j,k] * x[j,k,t] for j in DC for k in Cust for t in Period) +
    quicksum(run_dc_cost[j] * y[j,t] for j in DC for t in Period)
    ,GRB.MINIMIZE
)
model.update()

In [140]:
model.optimize()

Optimize a model with 24 rows, 24 columns and 54 nonzeros
Variable types: 18 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 20 rows and 18 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 9 nonzeros
Variable types: 3 continuous, 3 integer (3 binary)

Root relaxation: objective 2.600000e+03, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2600.0000000 2600.00000  0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 2600 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.600000000000e+03, best bound 2.600000000000e+03, gap 0.0000%


In [141]:
for v in model.getVars():
    print(v.VarName, v.X)

x[1,1,1] 0.0
x[1,1,2] 0.0
x[1,2,1] 0.0
x[1,2,2] 0.0
x[1,3,1] 0.0
x[1,3,2] 0.0
x[2,1,1] 0.0
x[2,1,2] 0.0
x[2,2,1] 0.0
x[2,2,2] 0.0
x[2,3,1] 0.0
x[2,3,2] 0.0
x[3,1,1] 10.0
x[3,1,2] 10.0
x[3,2,1] 10.0
x[3,2,2] 10.0
x[3,3,1] 10.0
x[3,3,2] 10.0
y[1,1] 0.0
y[1,2] -0.0
y[2,1] 0.0
y[2,2] -0.0
y[3,1] 1.0
y[3,2] 1.0
